# Load the database

In [11]:
import pandas as pd

datafile = 'Binance_BTCUSDT_1h.csv'
df = pd.read_csv(datafile,header=(1))
df.head()

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount
0,1677884400000,2023-03-03 23:00:00,BTCUSDT,22320.99,22377.26,22313.17,22354.34,6800.13647,1.519418e+08,225360
1,1677880800000,2023-03-03 22:00:00,BTCUSDT,22239.92,22338.75,22224.68,22320.99,5433.81752,1.210379e+08,212620
2,1677877200000,2023-03-03 21:00:00,BTCUSDT,22307.08,22316.15,22147.00,22239.92,14773.22101,3.286938e+08,374089
3,1677873600000,2023-03-03 20:00:00,BTCUSDT,22357.83,22378.22,22274.00,22308.16,8238.69603,1.839990e+08,251722
4,1677870000000,2023-03-03 19:00:00,BTCUSDT,22313.20,22397.76,22289.84,22358.48,8452.36394,1.887409e+08,270570


In [43]:
df.shape

(48397, 11)

In [44]:
# We will predict the next hour close price
predictionHours = 1
# Create another column shifted 'n'  units up
df['Prediction'] = df[['Close']].shift(predictionHours)
# show the first 5 rows
df.head()

,Unix,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USDT,tradecount,Prediction
1,1677880800000,2023-03-03 22:00:00,BTCUSDT,22239.92,22338.75,22224.68,22320.99,5433.81752,1.210379e+08,212620,NaN
2,1677877200000,2023-03-03 21:00:00,BTCUSDT,22307.08,22316.15,22147.00,22239.92,14773.22101,3.286938e+08,374089,22320.99
3,1677873600000,2023-03-03 20:00:00,BTCUSDT,22357.83,22378.22,22274.00,22308.16,8238.69603,1.839990e+08,251722,22239.92
4,1677870000000,2023-03-03 19:00:00,BTCUSDT,22313.20,22397.76,22289.84,22358.48,8452.36394,1.887409e+08,270570,22308.16
5,1677866400000,2023-03-03 18:00:00,BTCUSDT,22413.38,22422.91,22229.25,22313.20,14044.11611,3.138645e+08,366559,22358.48


In [45]:
#Removing the first row and the columns that we will not use
df.dropna(inplace = True)
bitcoin_data = df.drop(['Date','Symbol','Volume USDT'],axis = 1)
bitcoin_data.head()

,Unix,Open,High,Low,Close,Volume BTC,tradecount,Prediction
2,1677877200000,22307.08,22316.15,22147.00,22239.92,14773.22101,374089,22320.99
3,1677873600000,22357.83,22378.22,22274.00,22308.16,8238.69603,251722,22239.92
4,1677870000000,22313.20,22397.76,22289.84,22358.48,8452.36394,270570,22308.16
5,1677866400000,22413.38,22422.91,22229.25,22313.20,14044.11611,366559,22358.48
6,1677862800000,22375.36,22448.25,22366.26,22413.17,10207.73624,294309,22313.20


In [46]:
import numpy as np
import matplotlib.pyplot as plt
from time import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score
# regression models
from sklearn.linear_model import LinearRegression, Ridge,Lasso
# split X and y data
X = bitcoin_data.iloc[:, :-1].values
y = bitcoin_data.iloc[:, -1].values

In [47]:
# Scale the X data
sc = StandardScaler()
X_sc = sc.fit_transform(X)

In [48]:
#Splitting the data into train and test split
X_train, X_test, y_train, y_test = train_test_split(X_sc, y, test_size=0.2, random_state=42)

# Testing the regressors

In [42]:
regressors = [
    LinearRegression(),
    Lasso(),
    Ridge()
]

In [49]:
head = 10
for model in regressors[:head]:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    y_pred = model.predict(X_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, y_pred))
    print("\tMean absolute error:", mean_absolute_error(y_test, y_pred))
    print("\tR2 score:", r2_score(y_test, y_pred))
    print()

LinearRegression()
	Training time: 0.041s
	Prediction time: 0.009s
	Explained variance: 0.999838617167731
	Mean absolute error: 103.92317415528787
	R2 score: 0.9998386164152514

Lasso()
	Training time: 1.743s
	Prediction time: 0.001s
	Explained variance: 0.9997018696247526
	Mean absolute error: 141.92531002148527
	R2 score: 0.9997018664193504

Ridge()
	Training time: 0.031s
	Prediction time: 0.001s
	Explained variance: 0.9998333585564755
	Mean absolute error: 104.89656921993337
	R2 score: 0.9998333580703256



/home/otitamario/data-science/myprojects/bitcoin-predictor/.venv/lib/python3.10/site-packages/sklearn/linear_model/_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.243e+09, tolerance: 1.062e+09
  model = cd_fast.enet_coordinate_descent(
